## Analyse the run

In [ ]:
%matplotlib inline
import likelihoodfree.io as io
import likelihoodfree.viz as viz
import matplotlib.pyplot as plt
import numpy as np
import os

dirs = {}
dirs['dir_data'] = '../results/mog/data/'
dirs['dir_nets'] = '../results/mog/nets/'
dirs['dir_plots'] = '../results/mog/plots/'

prefix = 'plain'
postfix = 'iter_2'

## ... show some info on iteration 1

In [ ]:
info = io.load(dirs['dir_nets'] + prefix + '_' + postfix + '_info.pkl')
print(viz.info(info, title='{} {}'.format(prefix, postfix)))

## ... plot the training loss

In [ ]:
fig, ax = viz.loss(info)
plt.show()

## ... and the distance of samples to the observation

In [ ]:
dist = io.load(dirs['dir_nets'] + '/' + prefix + '_' + postfix + '_dist.pkl')
fig, ax = viz.dist(dist)

## ... what about the true posterior?

In [ ]:
sim = io.load(dirs['dir_nets'] + '/' + prefix + '_' + postfix + '_sim.pkl')

In [ ]:
true_posterior = sim.calc_posterior()

In [ ]:
x = np.atleast_2d(np.linspace(-7., 7., 1000)).T
plt.plot(x, true_posterior.eval(x, log=False), '-r')
plt.gca().set_xlabel(r'$\theta$')
plt.gca().set_ylabel(r'$p(\theta|x=x_0)$')
plt.show()

## ... what about the approximate posterior

In [ ]:
approx_posterior = io.load(dirs['dir_nets'] + '/' + prefix + '_' + postfix + '_posterior.pkl')

In [ ]:
shared = {}
shared['diag_only'] = True
shared['diag_only_rows'] = 1
shared['diag_only_cols'] = 2
fig, ax = viz.plot_pdf_marginals(true_posterior, lims=[-4.5, 4.5], **shared)
viz.plot_pdf_marginals(approx_posterior, lims=[-4.5, 4.5], ax=ax, **shared)
plt.show()

## ... at different $x_o$

In [ ]:
net = io.load(dirs['dir_nets'] + '/' + prefix + '_' + postfix + '_net.pkl')

# grids for plotting
xx = np.linspace(-12.0, 12.0, 100)
yy = np.linspace(-10.0, 10.0, 100)
X, Y = np.meshgrid(xx, yy)
xy = np.stack([X.flatten(), Y.flatten()], axis=1)

# show mdn's conditional probability density
fig, ax = plt.subplots(1, 1, facecolor='white')
Z = np.exp(net.eval([xy[:, 0:1], xy[:, 1:2]])).reshape(list(X.shape))
Z[Z>1]=1.  # clipping
ax.contour(X, Y, Z)
ax.set_xlabel('x')
ax.set_ylabel('$\mu$')
ax.set_title('$p(\mu|x)$')

plt.show()

## ... evaluating parameters

In [ ]:
xx = np.linspace(-12.0, 12.0, 100)

# show mdn's parameters
a, m, s = net.eval_comps(xx[:, np.newaxis])

cols = ['r', 'b']
# mixing coefficients
plt.close()
fig, ax = plt.subplots(2, net.n_components+1, facecolor='white')
for i in range(net.n_components):
    ax[0,0].plot(xx, a[:, i], color=cols[i], label='component {0}'.format(i))
ax[0,0].set_ylim([-0.2, 1.2])
ax[0,0].set_xlabel('x')
ax[0,0].set_ylabel('mixing coefficient')
ax[0,0].legend()

ax[0,1].set_axis_off()

# means and stds
for i in range(net.n_components):
    m[i] = np.squeeze(m[i])
    s[i] = np.squeeze(s[i])
    ax[1,i].plot(xx, m[i], '-', color=cols[i], label='m')
    ax[1,i].plot(xx, m[i] + s[i], ':', color=cols[i], label='m +/- s')
    ax[1,i].plot(xx, m[i] - s[i], ':', color=cols[i])
    ax[1,i].set_xlabel('x')
    ax[1,i].set_ylabel('$\mu$')
    ax[1,i].legend()

ax[0,2].set_axis_off()
ax[1,2].set_axis_off()